# CRF+LSTM

keras 2.2.4

tensorflow 1.13

pip install git+https://www.github.com/keras-team/keras-contrib.git

In [72]:
import re
import os
import pandas as pd

In [53]:
char_vocab_path = "CRF/data/char_vocabs.txt" # 字典文件
#train_data_path = 'data/train_data/train_data_000' # 训练数据
#train_data_path = './data/train_data' # 训练数据
#test_data_path = 'data/train_data/train_data_000' # 测试数据

special_words = ['<PAD>', '<UNK>'] # 特殊词表示

# "BIO"标记的标签
#label2idx = {"O": 0,
#             "B-PER": 1, "I-PER": 2,
#             "B-LOC": 3, "I-LOC": 4,
#             "B-ORG": 5, "I-ORG": 6
#            }
label2idx = {'O': 0,
             'B-DISEASE': 1, 'B-DISEASE_GROUP': 2,
             'B-DRUG_DOSAGE': 3, 'B-DRUG_EFFICACY': 4,
             'B-DRUG_INGREDIENT': 5, 'B-DRUG_TASTE': 6,
             'B-FOOD_GROUP':7, 'B-PERSON_GROUP':8,
             'B-SYMPTOM':9, 'B-SYNDROME':10,
             'I-DISEASE': 11, 'I-DISEASE_GROUP': 12,
             'I-DRUG_DOSAGE': 13, 'I-DRUG_EFFICACY': 14,
             'I-DRUG_INGREDIENT': 15, 'I-DRUG_TASTE': 16,
             'I-FOOD_GROUP':17, 'I-PERSON_GROUP':18,
             'I-SYMPTOM':19, 'I-SYNDROME':20
            }

# 索引和BIO标签对应
idx2label = {idx: label for label, idx in label2idx.items()}

# 读取字符词典文件
with open(char_vocab_path, "r", encoding="utf8") as fo:
    char_vocabs = [line.strip() for line in fo]
char_vocabs = special_words + char_vocabs

# 字符和索引编号对应
idx2vocab = {idx: char for idx, char in enumerate(char_vocabs)}
vocab2idx = {char: idx for idx, char in idx2vocab.items()}

In [54]:
# 读取训练语料
def read_corpus(corpus_path, vocab2idx, label2idx):
    with open(corpus_path, encoding='utf-8') as fr:
        lines = fr.readlines()

    sent_, tag_ = [], []
    for letter in lines:
        [char,label,_] = re.split('\t|\n',letter)
        sent_.append(char)
        tag_.append(label)

    sent_ids = [vocab2idx[char] if char in vocab2idx else vocab2idx['<UNK>'] for char in sent_]
    tag_ids = [label2idx[label] if label in label2idx else 0 for label in tag_]
    return sent_ids, tag_ids

# 加载训练集
#train_datas, train_labels = read_corpus(train_data_path, vocab2idx, label2idx)
# 加载测试集
#test_datas, test_labels = read_corpus(test_data_path, vocab2idx, label2idx)


In [55]:
train_datas = []
train_labels = []
files = os.listdir('data/train_data')
for file in files:
    train_data_path_i = 'data/train_data/'+file
    train_datas_i, train_labels_i = read_corpus(train_data_path_i, vocab2idx, label2idx)
    train_datas.append(train_datas_i)
    train_labels.append(train_labels_i)
    #if i%10==0:
    #    print(i)

In [56]:
valid_datas = []
valid_labels = []
files = os.listdir('data/valid_data')
for file in files:
    valid_data_path_i = 'data/valid_data/'+file
    valid_datas_i, valid_labels_i = read_corpus(valid_data_path_i, vocab2idx, label2idx)
    valid_datas.append(valid_datas_i)
    valid_labels.append(valid_labels_i)

In [57]:
print(train_datas[50])
print([idx2vocab[idx] for idx in train_datas[50]])
print(train_labels[50])
print([idx2label[idx] for idx in train_labels[50]])

[1, 61, 77, 1, 1, 17, 181, 3093, 3817, 2654, 6214, 1959, 2177, 286, 6802, 5965, 519, 1408, 2644, 2102, 2732, 1842, 889, 2545, 3093, 3817]
['<UNK>', 'b', 'r', '<UNK>', '<UNK>', '3', '、', '治', '疗', '期', '间', '忌', '房', '事', '，', '配', '偶', '如', '有', '感', '染', '应', '同', '时', '治', '疗']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [58]:
import numpy as np
import keras
from keras.models import Sequential
from keras.models import Model
from keras.layers import Masking, Embedding, Bidirectional, LSTM, Dense, Input, TimeDistributed, Activation
from keras.preprocessing import sequence
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_viterbi_accuracy
from keras import backend as K
K.clear_session()

EPOCHS = 50
BATCH_SIZE = 128
EMBED_DIM = 48
HIDDEN_SIZE = 16
MAX_LEN = 50
VOCAB_SIZE = len(vocab2idx)
CLASS_NUMS = len(label2idx)
print(VOCAB_SIZE, CLASS_NUMS)

print('padding sequences')
train_datas = sequence.pad_sequences(train_datas, maxlen=MAX_LEN)
train_labels = sequence.pad_sequences(train_labels, maxlen=MAX_LEN)
valid_datas = sequence.pad_sequences(valid_datas, maxlen=MAX_LEN)
valid_labels = sequence.pad_sequences(valid_labels, maxlen=MAX_LEN)
print('x_train shape:', train_datas.shape)
print('x_test shape:', valid_datas.shape)

train_labels = keras.utils.to_categorical(train_labels, CLASS_NUMS)
valid_labels = keras.utils.to_categorical(valid_labels, CLASS_NUMS)
print('trainlabels shape:', train_labels.shape)
print('testlabels shape:', valid_labels.shape)

## BiLSTM+CRF模型构建
inputs = Input(shape=(MAX_LEN,), dtype='int32')
x = Masking(mask_value=0)(inputs)
x = Embedding(VOCAB_SIZE, EMBED_DIM, mask_zero=True)(x)
x = Bidirectional(LSTM(HIDDEN_SIZE, return_sequences=True))(x)
x = TimeDistributed(Dense(CLASS_NUMS))(x)
outputs = CRF(CLASS_NUMS)(x)
model = Model(inputs=inputs, outputs=outputs)
model.summary()

model.compile(loss=crf_loss, optimizer='adam', metrics=[crf_viterbi_accuracy])
model.fit(train_datas, train_labels, epochs=EPOCHS, verbose=1, validation_split=0.1)

score = model.evaluate(valid_datas, valid_labels, batch_size=BATCH_SIZE)
print(model.metrics_names)
print(score)

6874 21
padding sequences
x_train shape: (6899, 50)
x_test shape: (1987, 50)
trainlabels shape: (6899, 50, 21)
testlabels shape: (1987, 50, 21)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 50)                0         
_________________________________________________________________
masking_1 (Masking)          (None, 50)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 50, 48)            329952    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 50, 32)            8320      
_________________________________________________________________
time_distributed_1 (TimeDist (None, 50, 21)            693       
_________________________________________________________________
crf_1 (CRF)                  (None, 50, 21)            945      

6209/6209 [==============================] - 24s 4ms/step - loss: 2.9196 - crf_viterbi_accuracy: 0.9436 - val_loss: 3.0596 - val_crf_viterbi_accuracy: 0.8938
Epoch 42/50
6209/6209 [==============================] - 19s 3ms/step - loss: 2.9177 - crf_viterbi_accuracy: 0.9454 - val_loss: 3.0559 - val_crf_viterbi_accuracy: 0.8945
Epoch 43/50
6209/6209 [==============================] - 17s 3ms/step - loss: 2.9172 - crf_viterbi_accuracy: 0.9450 - val_loss: 3.0553 - val_crf_viterbi_accuracy: 0.8987
Epoch 44/50
6209/6209 [==============================] - 16s 3ms/step - loss: 2.9157 - crf_viterbi_accuracy: 0.9464 - val_loss: 3.0587 - val_crf_viterbi_accuracy: 0.8969
Epoch 45/50
6209/6209 [==============================] - 16s 3ms/step - loss: 2.9149 - crf_viterbi_accuracy: 0.9485 - val_loss: 3.0553 - val_crf_viterbi_accuracy: 0.8962
Epoch 46/50
6209/6209 [==============================] - 16s 3ms/step - loss: 2.9139 - crf_viterbi_accuracy: 0.9489 - val_loss: 3.0592 - val_crf_viterbi_accuracy:

In [59]:
# save model
model.save("model/ch_ner_model.h5")

In [76]:
def get_valid_nertag(input_data, result_tags):
    result_words = []
    start, end =0, 1 # 实体开始结束位置标识
    tag_label = "O" # 实体类型标识
    number = 0
    for i, tag in enumerate(result_tags):
        if tag.startswith("B"):
            number += 1
            if tag_label != "O": # 当前实体tag之前有其他实体
                result_words.append(('T'+str(number), tag_label,start,end,input_data[start: end])) # 获取实体
            tag_label = tag.split("-")[1] # 获取当前实体类型
            start, end = i, i+1 # 开始和结束位置变更
        elif tag.startswith("I"):
            temp_label = tag.split("-")[1]
            if temp_label == tag_label: # 当前实体tag是之前实体的一部分
                end += 1 # 结束位置end扩展
        elif tag == "O":
            if tag_label != "O": # 当前位置非实体 但是之前有实体
                result_words.append(('T'+str(number), tag_label,start,end,input_data[start: end])) # 获取实体
                tag_label = "O"  # 实体类型置"O"
            start, end = i, i+1 # 开始和结束位置变更
    if tag_label != "O": # 最后结尾还有实体
        result_words.append(('T'+str(number),tag_label, start,end,input_data[start: end])) # 获取结尾的实体
    return result_words

In [77]:
maxlen = 50
result = {}
test_data_path = 'data/chusai_xuanshou/'
for i in range(500):
    test_file = test_data_path+str(i+1000)+'.txt'
    with open(test_file, "r", encoding="utf8") as test:
        sentence = test.read()
    sentences = sentence.split('。')
    y_ner = []

    for sent in sentences:
        sent = sent.replace(' ','_')
        sent_chars = list(sent+'。')
        sent2id = [vocab2idx[word] if word in vocab2idx else vocab2idx['<UNK>'] for word in sent_chars]

        sent2id_new = np.array([[0] * (maxlen-len(sent2id)) + sent2id[:maxlen]])
        y_pred = model.predict(sent2id_new)
        y_label = np.argmax(y_pred, axis=2)
        y_label = y_label.reshape(1, -1)[0]
        y_ner_ = [idx2label[i] for i in y_label][-len(sent_chars):]
        y_ner.extend(y_ner_)
    result_words = get_valid_nertag(sentence, y_ner)
    result[i+1000] = result_words
    for res in result_words:
        number = res[0]
        tag = res[1]
        start = res[2]
        end = res[3]
        word = res[4]
        
        print(i+1000,number, tag, start, end, "".join(word))

1000 T1 DRUG_EFFICACY 138 142 清热解毒
1000 T2 DRUG_EFFICACY 143 147 化湿除带
1000 T3 DRUG_EFFICACY 148 152 祛瘀止痛
1000 T4 DISEASE 160 165 慢性盆腔炎
1001 T1 DISEASE 36 39 糖尿病
1001 T2 SYMPTOM 87 91 月经量少
1001 T3 SYMPTOM 95 101 血虚萎黄后错
1001 T4 SYMPTOM 102 106 血虚萎黄
1001 T5 SYMPTOM 107 111 风湿痹痛
1001 T7 SYMPTOM 112 116 肢体麻木
1001 T7 DISEASE 116 119 糖尿病
1002 T1 PERSON_GROUP 36 42 感冒发热病人
1002 T2 DISEASE 51 54 高血压
1002 T3 DISEASE_GROUP 55 58 心脏病
1002 T4 DISEASE_GROUP 59 61 肝病
1002 T5 DISEASE_GROUP 66 68 肾病
1002 T6 SYMPTOM 90 94 月经紊乱
1002 T7 SYMPTOM 109 111 眩晕
1002 T8 PERSON_GROUP 152 155 过敏者
1002 T9 PERSON_GROUP 158 163 过敏体质者
1002 T10 PERSON_GROUP 192 194 儿童
1002 T11 DRUG_DOSAGE 242 244 颗粒
1002 T12 DRUG_TASTE 245 248 气微香
1002 T13 DRUG_TASTE 249 252 味微苦
1002 T14 SYMPTOM 282 286 烘热汗出
1002 T15 SYMPTOM 287 291 头晕耳鸣
1002 T16 SYMPTOM 297 301 五心烦热
1002 T17 SYMPTOM 302 306 腰背酸痛
1002 T18 SYMPTOM 307 311 大便干燥
1002 T19 SYMPTOM 312 315 心烦易
1002 T21 SYNDROME 387 389 2g
1002 T21 SYNDROME 389 391 *1
1003 T2 DISEASE 55 59 子宫肌

1026 T1 FOOD_GROUP 5 7 寒凉
1026 T2 FOOD_GROUP 8 10 生冷
1026 T3 PERSON_GROUP 15 17 孕妇
1026 T4 PERSON_GROUP 18 23 糖尿病患者
1026 T5 DISEASE 35 37 感冒
1026 T6 SYMPTOM 86 90 月经量少
1026 T7 SYMPTOM 92 96 月经错后
1026 T8 SYMPTOM 98 105 阴道不规则出血
1026 T9 PERSON_GROUP 164 169 过敏体质者
1026 T10 DRUG_EFFICACY 284 288 补血活血
1026 T11 SYNDROME 296 298 血虚
1026 T12 SYMPTOM 299 303 月经不调
1026 T14 SYMPTOM 304 306 痛经
1026 T14 PERSON_GROUP 306 308 儿童
1026 T15 DRUG_DOSAGE 312 316 薄膜衣片
1027 T1 DRUG_DOSAGE 8 10 颗粒
1027 T2 DRUG_TASTE 11 13 气香
1027 T3 DRUG_TASTE 14 16 味甜
1027 T4 DRUG_TASTE 17 18 辛
1027 T5 DRUG_TASTE 21 22 辛
1027 T6 SYMPTOM 75 77 血虚
1027 T7 SYMPTOM 79 83 月经不调
1027 T8 SYMPTOM 84 87 痛经 
1028 T1 PERSON_GROUP 29 31 孕妇
1028 T2 SYNDROME 53 57 湿热下注
1028 T3 SYMPTOM 58 62 白带过多
1028 T4 DRUG_DOSAGE 85 88 胶囊剂
1028 T5 DRUG_INGREDIENT 160 162 和皮
1028 T6 DRUG_INGREDIENT 163 166 真菌有
1028 T7 DRUG_EFFICACY 167 170 制作用
1028 T8 DRUG_INGREDIENT 207 209 勺有
1028 T9 DRUG_EFFICACY 210 214 血通瘀去
1029 T1 PERSON_GROUP 0 2 孕妇
1029 T2 SYNDROM

1045 T1 DRUG_EFFICACY 104 108 活血调经
1045 T2 DRUG_EFFICACY 109 113 去瘀止痛
1045 T3 SYMPTOM 118 122 瘀血不净
1045 T4 SYMPTOM 132 136 瘀血不净
1045 T5 DRUG_DOSAGE 145 148 胶囊剂
1045 T6 SYMPTOM 168 172 痢?/t
1045 T8 SYMPTOM 175 177 /t
1045 T9 DRUG_EFFICACY 177 181 r><t
1045 T9 DRUG_EFFICACY 181 185 r cl
1045 T10 DRUG_EFFICACY 186 190 ss='
1045 T11 SYMPTOM 195 199 Whit
1046 T1 DRUG_EFFICACY 4 8 补血健脑
1046 T2 DRUG_EFFICACY 43 46 补气血
1046 T3 SYMPTOM 58 60 头晕
1047 T1 DRUG_DOSAGE 4 8 薄膜衣片
1047 T2 DRUG_TASTE 23 26 气微香
1047 T4 PERSON_GROUP 102 104 宫平
1047 T4 SYNDROME 104 108 滑肌有一
1047 T5 SYMPTOM 113 115 ，并
1047 T6 SYMPTOM 116 118 小鼠
1047 T8 DISEASE 132 137 .32g 
1047 T8 SYMPTOM 137 139  用
1047 T9 SYMPTOM 140 142 妇人
1047 T10 SYMPTOM 143 150 血阻络所所致块
1047 T11 DRUG_DOSAGE 163 165 尽；
1048 T1 FOOD_GROUP 12 14 寒凉
1048 T2 FOOD_GROUP 15 17 生冷
1048 T3 DRUG_INGREDIENT 41 43 藜芦
1048 T4 DRUG_INGREDIENT 44 47 五灵脂
1048 T5 DRUG_INGREDIENT 48 50 皂荚
1048 T6 PERSON_GROUP 56 61 糖尿病患者
1048 T7 SYMPTOM 71 75 月经提前
1048 T8 DISEASE 81 83

1073 T1 SYMPTOM 66 70 用于更年
1073 T2 SYMPTOM 71 75 潮热汗出
1073 T3 SYNDROME 83 87 失眠 尚
1073 T4 SYMPTOM 153 157 用于更年
1073 T5 SYMPTOM 158 164 潮热汗出，眩
1073 T6 DRUG_EFFICACY 175 185 粒装0.3g  1.
1073 T7 DRUG_EFFICACY 186 190 善心脑血
1073 T8 DRUG_EFFICACY 199 203 律，降低
1073 T9 DRUG_EFFICACY 204 208 压。2.
1073 T10 DRUG_EFFICACY 211 216 神系统安神
1073 T11 DRUG_EFFICACY 218 222 ，改善睡
1073 T12 DRUG_EFFICACY 223 228 。3.调节
1073 T13 SYMPTOM 253 257 系统质量
1073 T14 DRUG_EFFICACY 260 264 骨质疏松
1073 T15 DRUG_EFFICACY 296 300 和改善作
1074 T1 DISEASE_GROUP 79 80 淋
1074 T2 DRUG_EFFICACY 84 88 淋巴肿大
1074 T3 DRUG_EFFICACY 89 91 疼痛
1075 T1 DRUG_EFFICACY 48 52 补阴培阳
1075 T2 DRUG_EFFICACY 53 57 益肾健脾
1075 T3 DRUG_EFFICACY 87 91 补阴培阳
1075 T4 DRUG_EFFICACY 92 97 益肾健脾 
1076 T1 DRUG_INGREDIENT 84 86 抗力
1076 T2 DRUG_INGREDIENT 87 89 香附
1076 T3 DRUG_INGREDIENT 90 92 延胡
1076 T4 DRUG_EFFICACY 93 99 赤芍能抑制子
1076 T5 SYMPTOM 127 131 疗妇女月
1076 T6 SYMPTOM 132 134 不调
1076 T7 DRUG_EFFICACY 144 148 。  养
1076 T8 DRUG_EFFICACY 149 153 舒肝，调
1076 T10 SYND

1103 T1 DRUG_EFFICACY 13 17 促进造血
1103 T2 DRUG_EFFICACY 18 20 止血
1103 T3 DRUG_DOSAGE 344 347 致腹腔
1103 T4 DRUG_TASTE 348 350 出液
1103 T5 DRUG_TASTE 351 353 白细
1103 T6 PERSON_GROUP 366 368 物（
1103 T7 DRUG_INGREDIENT 412 414 微苦
1103 T8 DRUG_INGREDIENT 415 418   6
1103 T9 DRUG_INGREDIENT 419 421 *1
1103 T10 DISEASE 448 451 本药时
1103 T11 DISEASE_GROUP 452 455 宜同时
1103 T12 SYMPTOM 636 640 在儿童不
1103 T14 SYMPTOM 641 647 接触的地方。
1103 T14 DRUG_EFFICACY 647 651 12.如
1103 T15 SYMPTOM 659 663 ，使用本
1103 T16 SYMPTOM 664 668 前请咨询
1103 T17 SYMPTOM 669 673 师或药师
1103 T18 SYMPTOM 674 678   6g
1103 T19 SYMPTOM 679 683 12袋。
1103 T20 DRUG_INGREDIENT 749 751 次6
1103 T21 DRUG_INGREDIENT 752 754 ，一
1103 T22 DRUG_INGREDIENT 755 758 2次。
1103 T23 DRUG_INGREDIENT 762 764 服用
1103 T24 DRUG_INGREDIENT 776 778 药师
1103 T25 DRUG_INGREDIENT 779 782 包括任
1103 T26 DRUG_INGREDIENT 783 785 从药
1103 T27 DRUG_INGREDIENT 786 788 、超
1103 T28 DRUG_INGREDIENT 789 791 或保
1103 T29 DRUG_INGREDIENT 795 797 购买
1103 T30 DRUG_INGREDIENT 798 800

...

1412 T1 SYNDROME 30 34 气血两虚
1412 T3 SYNDROME 35 37 气滞
1412 T3 SYNDROME 37 39 血瘀
1412 T4 SYMPTOM 42 46 月经不调
1412 T5 SYMPTOM 49 53 月经提前
1412 T6 SYMPTOM 54 58 月经错后
1412 T7 SYMPTOM 59 63 月经量多
1412 T8 SYMPTOM 64 68 神疲乏力
1412 T10 SYMPTOM 69 73 行经腹痛
1412 T10 DRUG_EFFICACY 73 77 益气养血
1412 T11 DRUG_DOSAGE 78 81 理气活
1413 T1 SYNDROME 3 7 气血两虚
1413 T2 SYMPTOM 10 14 月经不调
1413 T3 SYMPTOM 30 34 月经不调
1413 T4 SYMPTOM 37 43 月经周期错后
1413 T5 SYMPTOM 44 48 行经量少
1413 T6 DRUG_EFFICACY 57 61 力  如
1413 T7 SYNDROME 64 68 药物同时
1413 T8 SYMPTOM 82 88 详情请咨询医
1413 T9 SYMPTOM 89 93 或药师。
1413 T10 SYMPTOM 94 98  益气养
1413 T11 SYMPTOM 99 103 ，活血调
1414 T1 DRUG_EFFICACY 1 4 补气血
1414 T2 DRUG_EFFICACY 5 8 调月经
1414 T3 SYMPTOM 11 15 月经不调
1414 T4 DRUG_EFFICACY 39 42 补气血
1414 T5 DRUG_EFFICACY 43 46 调月经
1414 T6 SYMPTOM 49 53 头晕心慌
1414 T7 SYMPTOM 54 58 疲乏无力
1414 T8 SYMPTOM 59 63 月经量少
1415 T1 DRUG_EFFICACY 41 45 活血调经
1415 T2 SYNDROME 48 52 气血两虚
1415 T3 SYMPTOM 66 72 月经周期错后
1415 T4 SYMPTOM 73 77 行经量少
1415 T5 SYMPTOM 78 82 精神不振
1415 T

1435 T1 DRUG_EFFICACY 1 4 补气血
1435 T2 DRUG_EFFICACY 5 8 调月经
1435 T3 DRUG_EFFICACY 45 49 活血调经
1435 T4 SYNDROME 52 56 气血两虚
1435 T5 SYMPTOM 70 76 月经周期错后
1435 T6 SYMPTOM 77 81 行经量少
1435 T7 SYMPTOM 82 86 精神不振
1435 T9 SYMPTOM 97 101 月经不调
1435 T10 SYNDROME 101 107 血瘀气血两虚
1435 T10 DRUG_EFFICACY 107 111 益气养血
1436 T1 DRUG_EFFICACY 15 19 调经止痛
1436 T3 SYNDROME 22 24 血虚
1436 T3 SYNDROME 24 26 肝郁
1436 T4 SYMPTOM 28 32 月经不调
1436 T5 SYMPTOM 33 35 痛经
1436 T6 SYMPTOM 50 54 经水量少
1436 T7 SYMPTOM 55 58 有血块
1436 T8 SYMPTOM 59 65 行经小腹疼痛
1436 T10 SYMPTOM 66 68 血块
1436 T10 SYMPTOM 68 70 排出
1436 T12 SYNDROME 72 74 ，经
1436 T12 SYNDROME 74 76 前双
1436 T13 SYMPTOM 78 82 痛调经止
1436 T14 SYMPTOM 83 85 。用
1436 T15 SYMPTOM 100 104 月经前后
1436 T16 SYMPTOM 105 108 证，症
1436 T17 SYMPTOM 109 115 行经后错，经
1436 T19 SYMPTOM 116 118 量少
1436 T19 SYMPTOM 118 120 、有
1436 T20 DRUG_INGREDIENT 225 227 方由
1436 T21 DRUG_INGREDIENT 228 230 物汤
1436 T22 DRUG_INGREDIENT 231 233 味而
1436 T23 DRUG_EFFICACY 234 240 。现代药理研
1436 T24 SYMPTOM 268 272 证，

1453 T1 DRUG_EFFICACY 86 88 镇痛
1453 T2 SYMPTOM 91 98 子宫平滑肌收缩
1453 T3 DRUG_INGREDIENT 111 113 方中
1453 T4 DRUG_EFFICACY 115 119 艾叶具有
1453 T5 DRUG_EFFICACY 120 122 制纤
1453 T6 DRUG_EFFICACY 143 145 高痛
1453 T7 SYMPTOM 156 160 肌，抑制
1453 T8 SYMPTOM 189 193 并具有雌
1453 T9 SYMPTOM 194 196 素样
1453 T10 SYMPTOM 197 201 用。4.
1453 T11 SYMPTOM 202 204 临床
1453 T12 SYMPTOM 205 207 要用
1453 T13 SYNDROME 211 213 女月
1453 T14 SYMPTOM 224 231 ，崩漏，带下等
1453 T15 SYMPTOM 246 248 有降
1453 T16 SYMPTOM 249 253 肠管紧张
1453 T17 SYMPTOM 254 256 ，拮
1453 T18 DRUG_EFFICACY 273 277 性肠炎，
1453 T20 SYNDROME 280 282 病证
1453 T20 SYNDROME 282 284 有一
1453 T21 SYMPTOM 297 299 宫调
1453 T22 SYMPTOM 302 306 于血虚气
1453 T23 SYMPTOM 307 310 、下焦
1453 T24 SYMPTOM 311 314 寒所致
1453 T25 SYMPTOM 315 319 月经不调
1453 T26 SYMPTOM 320 328 痛经，症见行经后
1453 T27 SYMPTOM 342 346 、经行小
1453 T28 SYMPTOM 359 363 错、经量
1453 T29 SYMPTOM 364 366 、有
1453 T30 SYMPTOM 367 371 块、小腹
1453 T31 SYMPTOM 372 374 痛经
1453 T32 SYMPTOM 375 377 少理
1453 T33 SYMPTOM 385 392 。用于血虚气滞
145

1475 T1 DISEASE_GROUP 21 23   
1475 T2 SYMPTOM 50 54 月经量少
1475 T3 SYMPTOM 58 64 经来腹痛后错
1475 T4 SYMPTOM 65 69 经来腹痛
1475 T6 SYMPTOM 149 153 经水量少
1475 T6 DRUG_EFFICACY 153 157 活血调经
1476 T1 SYNDROME 64 66 血瘀
1476 T2 SYMPTOM 69 73 月经不调
1476 T4 SYMPTOM 76 82 经水量少血瘀
1476 T4 DRUG_EFFICACY 82 87 活血调经 
1476 T5 DRUG_EFFICACY 106 110 祛瘀生新
1476 T6 SYMPTOM 113 117 月经量少
1476 T8 SYMPTOM 121 125 经来腹痛
1476 T8 SYMPTOM 125 129 月经量少
1476 T9 SYMPTOM 133 137 经来腹痛
1477 T1 SYMPTOM 21 25 月经量少
1477 T2 SYMPTOM 29 33 经来腹痛
1477 T3 SYNDROME 71 73 公司
1477 T4 SYMPTOM 76 80 血调经。
1477 T6 SYMPTOM 83 87 瘀所致的
1477 T7 DRUG_EFFICACY 87 92 月经不调，
1477 T7 PERSON_GROUP 92 94 症见
1478 T1 DRUG_EFFICACY 7 10 补气血
1478 T2 DRUG_EFFICACY 11 14 调月经
1478 T3 SYNDROME 19 23 气血两虚
1478 T4 SYMPTOM 37 41 月经不调
1478 T5 SYMPTOM 44 50 月经周期错后
1478 T6 SYMPTOM 51 55 行经量少
1478 T7 SYMPTOM 56 60 精神不振
1478 T8 SYMPTOM 61 65 肢体乏力
1478 T9 FOOD_GROUP 120 122   
1478 T10 FOOD_GROUP 123 125 津中
1478 T11 PERSON_GROUP 130 136 股份有限公司
1478 T12 DISEASE 144 147 9g*
14

1496 T1 DRUG_EFFICACY 38 40 镇痛
1496 T2 SYMPTOM 45 50 平滑肌收缩
1496 T3 DRUG_INGREDIENT 67 69 艾叶
1496 T4 DRUG_EFFICACY 71 75 抑制纤溶
1496 T5 DRUG_EFFICACY 76 78 抗炎
1496 T6 DRUG_EFFICACY 92 94 镇痛
1496 T7 DRUG_EFFICACY 103 109 抑制子宫收缩
1496 T8 SYMPTOM 145 149 .故临床
1496 T9 SYMPTOM 150 152 要用
1496 T10 SYMPTOM 153 157 治疗妇女
1496 T11 SYMPTOM 158 160 经不
1496 T12 SYMPTOM 161 163 ，痛
1496 T13 SYNDROME 167 169 不孕
1496 T14 SYMPTOM 180 187 寒证型者。此外
1496 T15 SYMPTOM 202 204 乙酰
1496 T16 SYMPTOM 205 209 碱的作用
1496 T17 SYMPTOM 210 212 临床
1496 T19 SYNDROME 226 228 证有
1496 T19 SYNDROME 228 230 一定
1496 T20 SYMPTOM 238 242 血虚气滞
1496 T21 DRUG_EFFICACY 252 256 不调、痛
1496 T23 SYNDROME 259 261 理气
1496 T23 SYNDROME 261 263 养血
1496 T24 SYMPTOM 276 278 下焦
1496 T25 SYMPTOM 281 285 致的月经
1496 T26 SYMPTOM 286 289 调、痛
1496 T27 SYMPTOM 290 293 ，症见
1496 T28 SYMPTOM 294 298 经后错、
1496 T29 SYMPTOM 299 307 量少、有血块、小
1497 T1 SYNDROME 67 71 气血两虚
1497 T3 SYMPTOM 88 92 行经量少
1497 T3 SYMPTOM 92 98 月经周期错后
1497 T4 SYMPTOM 99 103 行经量少
1497 T5 SYMP

In [80]:
for i in range(1000,1500):
    pd.DataFrame(result[i]).to_csv('data/output_data/%d.ann'%i,
                                      sep=' ',
                                      header = None,
                                      index = 0,
                                      encoding = 'utf-8')

In [24]:
from keras.models import load_model
import numpy as np

maxlen = 40
sentence = " 北京同仁堂科技发展股份有限公司制药厂  1.忌食辛辣，少进油腻。 2.感冒发热病人不宜服用。 3.有高血压、心脏病、肝病、糖尿病、肾病等慢性病严重者应在医师指导下服用。 4.伴有月经紊乱者，应在医师指导下服用。 5.眩晕症状较重者，应及时去医院就诊。 6.服药2周症状无缓解，应去医院就诊。 7.对本品过敏者禁用，过敏体质者慎用。 8.本品性状发生改变时禁止使用。 9.请将本品放在儿童不能接触的地方。 10.如正在使用其他药品，使用本品前请咨询医师或药师。  本品为浅黄色至棕黄色颗粒，气微香，味微苦。  滋养肝肾、宁心安神。用于更年期综合症属阴虚肝旺症，症见烘热汗出，头晕耳鸣，失眠多梦，五心烦热，腰背酸痛，大便干燥，心烦易怒，舌红少苔，脉弦细或弦细 开水冲服。一次1袋(12g)，一日3次。  如与其他药物同时使用可能会发生药物相互作用，详情请咨询医师或药师。  12g*10袋/盒  用于更年期综合症属阴虚肝旺症  铝塑复合膜包装，每袋装12克，每盒装10袋。  非处方药物（甲类）,中药保护品种二级  12g*10袋/盒  用于更年期综合症属阴虚肝旺更年期综合症气微香，味微苦。"
#sentence = sentence.replace(' ','_')
sentences = sentence.split('。')
y_ner = []

for sent in sentences:
    sent = sent.replace(' ','_')
    sent_chars = list(sent+'。')
    sent2id = [vocab2idx[word] if word in vocab2idx else vocab2idx['<UNK>'] for word in sent_chars]

    sent2id_new = np.array([[0] * (maxlen-len(sent2id)) + sent2id[:maxlen]])
    y_pred = model.predict(sent2id_new)
    y_label = np.argmax(y_pred, axis=2)
    y_label = y_label.reshape(1, -1)[0]
    y_ner_ = [idx2label[i] for i in y_label][-len(sent_chars):]
    y_ner.extend(y_ner_)

In [27]:
result_words = get_valid_nertag(sentence, y_ner)
for (word, tag) in result_words:
    print("".join(word), tag)

感冒 DISEASE
发热 SYMPTOM
高血压 DISEASE
心脏病 DISEASE_GROUP
肝病 DISEASE_GROUP
糖尿病 DISEASE
肾病 DISEASE_GROUP
慢性病 DISEASE_GROUP
月经紊乱 SYMPTOM
眩晕 SYMPTOM
过敏者 PERSON_GROUP
儿童 PERSON_GROUP
颗粒 DRUG_DOSAGE
气微香 DRUG_TASTE
味微苦 DRUG_TASTE
  滋养肝肾 DRUG_EFFICACY
宁心安神 DRUG_EFFICACY
烘热汗出 SYMPTOM
头晕耳鸣 SYMPTOM
失眠多梦 SYMPTOM
五心烦热 SYMPTOM
腰背酸 SYMPTOM


In [25]:
list(zip(list(sentence),y_ner))

[(' ', 'O'),
 ('北', 'O'),
 ('京', 'O'),
 ('同', 'O'),
 ('仁', 'O'),
 ('堂', 'O'),
 ('科', 'O'),
 ('技', 'O'),
 ('发', 'O'),
 ('展', 'O'),
 ('股', 'O'),
 ('份', 'O'),
 ('有', 'O'),
 ('限', 'O'),
 ('公', 'O'),
 ('司', 'O'),
 ('制', 'O'),
 ('药', 'O'),
 ('厂', 'O'),
 (' ', 'O'),
 (' ', 'O'),
 ('1', 'O'),
 ('.', 'O'),
 ('忌', 'O'),
 ('食', 'O'),
 ('辛', 'O'),
 ('辣', 'O'),
 ('，', 'O'),
 ('少', 'O'),
 ('进', 'O'),
 ('油', 'O'),
 ('腻', 'O'),
 ('。', 'O'),
 (' ', 'O'),
 ('2', 'O'),
 ('.', 'O'),
 ('感', 'B-DISEASE'),
 ('冒', 'I-DISEASE'),
 ('发', 'B-SYMPTOM'),
 ('热', 'I-SYMPTOM'),
 ('病', 'O'),
 ('人', 'O'),
 ('不', 'O'),
 ('宜', 'O'),
 ('服', 'O'),
 ('用', 'O'),
 ('。', 'O'),
 (' ', 'O'),
 ('3', 'O'),
 ('.', 'O'),
 ('有', 'O'),
 ('高', 'B-DISEASE'),
 ('血', 'I-DISEASE'),
 ('压', 'I-DISEASE'),
 ('、', 'O'),
 ('心', 'B-DISEASE_GROUP'),
 ('脏', 'I-DISEASE_GROUP'),
 ('病', 'I-DISEASE_GROUP'),
 ('、', 'O'),
 ('肝', 'B-DISEASE_GROUP'),
 ('病', 'I-DISEASE_GROUP'),
 ('、', 'O'),
 ('糖', 'B-DISEASE'),
 ('尿', 'I-DISEASE'),
 ('病', 'I-DISEASE'),
 ('、'